In [1]:
# !pip3 install datasets transformers sentencepiece

In [2]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

/home/ai-server/anaconda3/envs/hmp_torch/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/ai-server/anaconda3/envs/hmp_torch/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [3]:
import pandas as pd
pd.set_option('colwidth',0)
df = pd.read_csv("train_corpus.txt", names = ["text", "label"])
df.head()

,text,label
0,မ ကျေနပ်မှု တွေ ကို တော့ ရင် ထဲ သိုဝှက် ထား တယ်,0
1,အခက်အခဲ ဆို တဲ့ စကား ကို ဘယ်သူ မှ မ ရှောင် နိုင် ခဲ့ သလို ပင်ပန်းခြင်း ဆို တဲ့ စကား ကို လည်း ဘယ်သူ မှ မ ပုန်းဝှက် နိုင် ခဲ့ ဘူး,1
2,တစ်နေ့တာ ရဲ့ နွမ်းနယ်မှု တွေ နဲ့ အိပ်ရာ ဝင် လည်း နောင် တစ်နေ့တာ အတွက် အားမွေး ရ ပြန် တယ်,0
3,စကား အပြော များ တော့ လူတတ် ကြီး လုပ် တယ် လို့ ဆို ကြ တယ်,0
4,စကား နည်း တော့ လည်း မာန ကြီး တယ် လို့ ဆို ပြန် တယ်,1


In [4]:
df['label'] = df['label'].fillna(0)
df['label'] = df['label'].astype(int)

In [5]:
len(df)

11080

In [6]:
train_df = df[0:9999]
test_df = df[10000:11080]

In [7]:
train_df

,text,label
0,မ ကျေနပ်မှု တွေ ကို တော့ ရင် ထဲ သိုဝှက် ထား တယ်,0
1,အခက်အခဲ ဆို တဲ့ စကား ကို ဘယ်သူ မှ မ ရှောင် နိုင် ခဲ့ သလို ပင်ပန်းခြင်း ဆို တဲ့ စကား ကို လည်း ဘယ်သူ မှ မ ပုန်းဝှက် နိုင် ခဲ့ ဘူး,1
2,တစ်နေ့တာ ရဲ့ နွမ်းနယ်မှု တွေ နဲ့ အိပ်ရာ ဝင် လည်း နောင် တစ်နေ့တာ အတွက် အားမွေး ရ ပြန် တယ်,0
3,စကား အပြော များ တော့ လူတတ် ကြီး လုပ် တယ် လို့ ဆို ကြ တယ်,0
4,စကား နည်း တော့ လည်း မာန ကြီး တယ် လို့ ဆို ပြန် တယ်,1
...,...,...
9994,အန္တရာယ် အရှိဆုံး လမ်း က တော့ ဂုန်းဆင်းလမ်း မ ဟုတ် ဘူး ထောင်ခြောက် ဖြစ် တယ်,1
9995,အကြီးမားဆုံး ပျော်ရွှင် ခြင်း က ရယူ တာ မ ဟုတ် ဘူး ပေးကမ်း တာ ပဲ ဖြစ် တယ်,0
9996,အကောင်းဆုံး သော ချမ်းသာ ကြွယ်ဝ ခြင်း က ငွေကြေး မ ဟုတ် ဘူး ကျန်းမာရေး ပဲ ဖြစ် တယ်,0
9997,အရေး အကြီးဆုံး တောင်းဆု က တော့ နောင် ဘဝ မ ဟုတ် ဘူး အခု လက်ရှိ ဘဝ ဖြစ် တယ်,0


In [8]:
test_df

,text,label
10000,ကိုယ့် ရဲ့ စိတ်ရှုပ် စရာ တွေ ကို လဲ ခဏခဏ ထုတ် မ ပြော ပါ နဲ့,1
10001,ဘယ်သူ က မှ သင့် ကို ကြာကြာ သနား ကရုဏာ သက် မှာ မ ဟုတ် ပါ ဘူး,1
10002,သင့် ကို အသုံးမကျ တဲ့ သူ လို့ ပဲ ထင် ကြ ပါ လိမ့် မယ်,1
10003,ဘယ် အရာ ကို မဆို ကိုယ့် ကိုယ် ကို ခံ နိုင် ဖို့ လေ့ကျင့် ရ မယ်,0
10004,ဘယ်သူ က မှ သင့် ကို အမြဲ ကူညီ နိုင် မှာ မ ဟုတ် လို့ ပါ,1
...,...,...
11075,မှန် ပါ တယ် အရင် က တော့ အဲ့ဒီ လို ဘဲ လောကကြီး ကို ပြောင်းလဲ ဖို့ ကိုယ့် ပတ်ဝန်းကျင် က လူ တွေ ကို ပြောင်းလဲ ဖို့ ဆို တဲ့ စိတ်ကူး ယဉ် ခဲ့ ဖူး ပါ တယ်,1
11076,ဒါပေမယ့် တကယ့် လက်တွေ့ ဘဝ မှာ အဲ့ဒီ အရာ က အင်မတန် ခက်ခဲ တာ ကို တွေ့ ရ ပါ လိမ့် တယ်,1
11077,တစ်ချိန် လုံး ဆိုး လာ ခဲ့ တဲ့ သူ တစ် ယောက် ဟာ နောင်တ တွေ ရ ပြီး တော့ ပြောင်း လဲ သွား တယ် ဆိုတာ ဇာတ်ကား တွေ ထဲ မှာ လောက် ပဲ ရှိ ပါ တယ်,0
11078,အပြင် မှာ က တော့ သူတို့ တွေ ဟာ နောင်တ လဲ မ ရ ဘူး မှား မှန်း လဲ မ သိ ကြ ဘူး,1


In [9]:
# Save the split datasets to text files
train_df.to_csv("train_nsp.csv", index=False)
test_df.to_csv("test_nsp.csv", index=False)

In [10]:
len(train_df), len(test_df)

(9999, 1080)

# Training the tokenizer

In [11]:
from tokenizers import SentencePieceBPETokenizer
from transformers import PreTrainedTokenizerFast

vocab_size = 30522
max_length = 256
truncate_longer_samples = True

with open("train_corpus.txt") as f:
    data = [line.rstrip().split(',') for line in f]

special_tokens = ["<pad>", "<unk>", "<cls>", "<sep>", "<mask>", "<S>", "<T>"]

tokenizer = SentencePieceBPETokenizer()
tokenizer.train_from_iterator(data, vocab_size=vocab_size, min_frequency=2, show_progress=True, special_tokens=special_tokens)

In [12]:
model_path = "pretrained-bert-nsp"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [13]:
# save the tokenizer
tokenizer.save_model(model_path)

['pretrained-bert-nsp/vocab.json', 'pretrained-bert-nsp/merges.txt']

In [14]:
fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer._tokenizer,
    model_max_length=max_length, truncation=True
)
fast_tokenizer.add_special_tokens({'pad_token': '<pad>','unk_token':'<unk>','cls_token':'<cls>','sep_token':'<sep>','mask_token':'<mask>'})

0

In [15]:
output = fast_tokenizer.encode("နေကောင်းလား တကယ် လား")

In [16]:
output

[3083, 519, 302, 377]

# Tokenizing the dataset

In [17]:
# Load the datasets
training_dataset = load_dataset('csv', data_files={'train': 'train_nsp.csv'})
testing_dataset = load_dataset('csv', data_files={'test': 'test_nsp.csv'})

# Tokenize the NSP datasets
def encode_for_nsp(examples):
    return fast_tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_special_tokens_mask=True
    )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [18]:
training_dataset['train'], testing_dataset['test']

(Dataset({
     features: ['text', 'label'],
     num_rows: 9999
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 1080
 }))

In [19]:
train_dataset = training_dataset["train"].map(encode_for_nsp, batched=True)
test_dataset = testing_dataset["test"].map(encode_for_nsp, batched=True)

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

In [20]:
train_dataset[0], test_dataset[0]

({'text': 'မ ကျေနပ်မှု တွေ ကို တော့ ရင် ထဲ သိုဝှက် ထား တယ်',
  'label': 0,
  'input_ids': [78,
   5016,
   97,
   88,
   112,
   165,
   226,
   4396,
   212,
   98,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,


In [21]:
# # Set the format for NSP datasets
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [22]:
train_dataset[1]

{'label': tensor(1),
 'input_ids': tensor([ 639,  126,  103,  303,   88,  433,  104,   78,  899,  162,  161,  378,
         1137,  391,  126,  103,  303,   88,  231,  433,  104,   78, 3062, 1811,
          162,  161,  139,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [23]:
test_dataset[1]

{'label': tensor(1),
 'input_ids': tensor([ 433,   74,  104,  224,   88, 1536, 1474, 1858,  617,  117,   78,  237,
          100,  139,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [24]:
len(train_dataset), len(test_dataset)

(9999, 1080)

# Loading the Model

In [25]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForNextSentencePrediction(config=model_config)

In [26]:
# Data Collator for NSP (No need for MLM collator)
data_collator = DataCollatorWithPadding(tokenizer=fast_tokenizer)

In [27]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,
    num_train_epochs=1,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=10,           # whether you don't have much space so you let only 10 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [29]:
# train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForNextSentencePrediction.forward` and have been ignored: special_tokens_mask, text. If special_tokens_mask, text are not expected by `BertForNextSentencePrediction.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,999
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 125
  Number of trainable parameters = 109,287,170
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=125, training_loss=0.7099315185546875, metrics={'train_runtime': 248.8192, 'train_samples_per_second': 40.186, 'train_steps_per_second': 0.502, 'total_flos': 1315423721272320.0, 'train_loss': 0.7099315185546875, 'epoch': 1.0})

# Using the Model

In [30]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

In [31]:
from tokenizers import SentencePieceBPETokenizer
max_length = 256

model_path = 'pretrained-bert-nsp'
# Replace these with the actual paths to your model files
vocab_file = model_path+"/vocab.json"
merges_file = model_path+"/merges.txt"

# Load the tokenizer
tokenizer = SentencePieceBPETokenizer(vocab_file, merges_file)

In [32]:
fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer._tokenizer,
    model_max_length=max_length, truncation=True
)
fast_tokenizer.add_special_tokens({'pad_token': '<pad>','unk_token':'<unk>','cls_token':'<cls>','sep_token':'<sep>','mask_token':'<mask>'})

4

In [33]:
from transformers import BertTokenizer, BertForNextSentencePrediction, pipeline
model = BertForNextSentencePrediction.from_pretrained(os.path.join(model_path, "checkpoint-35000"))

loading configuration file pretrained-bert-nsp/checkpoint-35000/config.json
Model config BertConfig {
  "architectures": [
    "BertForNextSentencePrediction"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 256,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.35.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pretrained-bert-nsp/checkpoint-35000/model.safetensors
All model checkpoint weights were used when initializing BertForNextSentencePrediction.

All the weights of BertForNextSentencePrediction were initialized from the model checkpoint at pretrained-bert-nsp/checkpoint-35000

In [34]:
sent1 = "စကား အပြော များ တော့ လူတတ် ကြီး လုပ် တယ် လို့ ဆို ကြ တယ်"
sent2 = "စကား အပြော နည်း တော့ လည်း မာန ကြီး တယ် လို့ ဆို ပြန် တယ်"

# Tokenize and encode test sentences
inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

# Perform inference
outputs = model(**inputs)
# Interpret predictions
is_next = outputs.logits[0, 0].item() > outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=True


In [35]:
sent1 = "ဒါပေမဲ့ လူ တွေ ကို ကြည့် တဲ့ အကြည့် က တော့ မျှတမှု မ ရှိ ဘူး"
sent2 = "လှလှ ထမင်းစား ပြီး ပြီ လား"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

# Interpret predictions
is_next= outputs.logits[0, 0].item() > outputs.logits[0, 1].item()

print(f"IsNext={is_next}")

IsNext=False


In [36]:
sent1 = "အောင်မြင် ဖို့ အတွက် ပဲ မ ကြိုးစား ပါ နဲ့"
sent2 = "တန်ဖိုး ရှိ တဲ့ လူ တစ်ယောက် ဖြစ် ဖို့ ကြိုးစား ပါ"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next = outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=True


In [37]:
sent1 = "ဟိုနေ့ က ကျွန်တော် ချော်လဲ တာ"
sent2 = "သင် မြင် တာ က သူတစ်ပါး ရဲ့ အပေါ်ယံ ပါ"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next = outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=False


In [38]:
sent1 = "စကား အပြော များ တော့ လူတတ် ကြီး လုပ် တယ် လို့ ဆို ကြ တယ်"
sent2 = "စကား အပြော နည်း တော့ လည်း မာန ကြီး တယ် လို့ ဆို ပြန် တယ်"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next= outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=True


In [39]:
sent1 = "ဒါပေမဲ့ လူ တွေ ကို ကြည့် တဲ့ အကြည့် က တော့ မျှတမှု မ ရှိ ဘူး"
sent2 = "စကား အပြော နည်း တော့ လည်း မာန ကြီး တယ် လို့ ဆို ပြန် တယ်"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next = outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=False


In [40]:
sent1 = "အဆင်ပြေ လား"
sent2 = "အဆင်ပြေ တယ် ပဲ ကြား ချင် ပါ တယ်"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next = outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=True


In [41]:
sent1 = "အဆင်ပြေ လား"
sent2 = "ကော်ဖီ ဖိတ်ကျ တယ် ဆိုတာ ခွက် ထဲ မှာ ကော်ဖီ ရှိ နေ လို့"

inputs = fast_tokenizer(sent1, sent2, return_tensors="pt")

outputs = model(**inputs)

is_next = outputs.logits[0, 0].item() >  outputs.logits[0, 1].item() 

print(f"IsNext={is_next}")

IsNext=False
